In [3]:
import tensorflow as tf
import pandas as pd

df=pd.read_csv(r'C:\Users\hamza jamil\Downloads\New folder (5)\train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
###Drop Nan Values
df=df.dropna()
X=df.drop('label',axis=1)
y=df['label']

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [11]:
messages=X.copy()

messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [15]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr


[[821, 1724, 3314, 3368, 3755, 1456, 2053, 4859, 4331, 2516],
 [171, 1584, 4456, 3655, 581, 3878, 525],
 [3637, 2436, 2843, 2088],
 [582, 3394, 2053, 142, 1672, 3876],
 [1640, 581, 4788, 3255, 3264, 3438, 581, 3455, 2991, 1361],
 [4101,
  2049,
  633,
  4145,
  3828,
  1901,
  1634,
  4367,
  4965,
  362,
  1227,
  2613,
  3237,
  3241,
  525],
 [1874, 3498, 900, 102, 296, 1811, 1895, 970, 3599, 630, 1302],
 [1855, 3984, 91, 2995, 4654, 3731, 1901, 2028, 3599, 630, 1302],
 [635, 90, 2359, 2005, 1904, 1929, 2865, 2992, 1901, 4412],
 [336, 415, 2018, 2646, 2960, 1517, 2880, 1104],
 [4113, 3170, 1359, 850, 204, 4968, 3308, 3747, 3109, 128, 369],
 [142, 210, 3755, 1929, 1901, 4654],
 [483, 466, 2163, 2261, 2389, 2698, 3754, 4564, 603],
 [1130, 4695, 4650, 979, 3714, 4600, 3492, 3599, 630, 1302],
 [2302, 582, 3279, 3099, 3705, 3599, 630, 1302],
 [3147, 3002, 2124, 542, 3538, 126, 2456, 4820, 3823, 311],
 [1669, 396, 1584],
 [2336, 1992, 386, 1769, 1901, 2763, 356, 525],
 [932, 2442, 4456, 3

# Embedding Representation

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4859 4331 2516]
 [   0    0    0 ...  581 3878  525]
 [   0    0    0 ... 2436 2843 2088]
 ...
 [   0    0    0 ... 3599  630 1302]
 [   0    0    0 ... 2663  918 1524]
 [   0    0    0 ... 1979 1030  114]]


In [17]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

W0522 02:50:30.870076 10668 deprecation.py:506] From C:\Users\hamza jamil\Anaconda3\lib\site-packages\tensorflow_core\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0522 02:50:31.022851 10668 deprecation.py:506] From C:\Users\hamza jamil\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0522 02:50:31.753728 10668 deprecation.py:323] From C:\Users\hamza jamil\Anaconda3\lib\site-packages\tensorflow_core\python\ops\nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is d

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
len(embedded_docs),y.shape

(18285, (18285,))

In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [21]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)


Train on 12250 samples, validate on 6035 samples
Epoch 1/5
12250/12250 [==============================] - 12s 993us/sample - loss: 0.3397 - acc: 0.8390 - val_loss: 0.1999 - val_acc: 0.9133
Epoch 2/5
12250/12250 [==============================] - 12s 944us/sample - loss: 0.1445 - acc: 0.9411 - val_loss: 0.2012 - val_acc: 0.9122
Epoch 3/5
12250/12250 [==============================] - 10s 785us/sample - loss: 0.1063 - acc: 0.9593 - val_loss: 0.2106 - val_acc: 0.9147
Epoch 4/5
12250/12250 [==============================] - 13s 1ms/sample - loss: 0.0806 - acc: 0.9691 - val_loss: 0.2607 - val_acc: 0.9145
Epoch 5/5
12250/12250 [==============================] - 12s 997us/sample - loss: 0.0630 - acc: 0.9776 - val_loss: 0.3031 - val_acc: 0.9032


# Adding Dropout

In [22]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
# Performance Metrics And Accuracy

In [25]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix

In [26]:
confusion_matrix(y_test,y_pred)

array([[1104, 2315],
       [1789,  827]], dtype=int64)

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.31996685998343